In [1]:
print("harshit")

harshit


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [3]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler, LabelEncoder , OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

In [4]:
##load the dtaset 
file=pd.read_csv("Churn_Modelling.csv")

In [5]:
file.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
#preprocessing the data 
## drop irrelevant column
file.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [7]:
file.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
##Encoding categorical dtaset 
label_encoder_gender=LabelEncoder()
file['Gender']=label_encoder_gender.fit_transform(file['Gender'])

In [9]:
file.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1


In [10]:
file['Gender'].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [11]:
file['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [12]:
#one hot encoding for geagraphy column
onehot_encoder=OneHotEncoder()

In [13]:
geo_encode=onehot_encoder.fit_transform(file[['Geography']])

In [14]:
geo_encode.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [15]:
geo_encode_df=pd.DataFrame(geo_encode.toarray(),columns=['geo_france','geo_germany','geo_spain'])

In [16]:
geo_encode_df

,geo_france,geo_germany,geo_spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [17]:
#combine one hot encoded columns with the original dataset
file=pd.concat([file.drop('Geography',axis=1),geo_encode_df],axis=1)

In [18]:
file.head(3)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,geo_france,geo_germany,geo_spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0


In [19]:
## save the encoder and scaler 
with open('label_encoder_gender.pkl','wb') as f:
   pickle.dump(label_encoder_gender,f)

with open('onehot_encoder.pkl','wb') as f:
    pickle.dump(onehot_encoder,f)

In [20]:
file.head(3)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,geo_france,geo_germany,geo_spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0


In [21]:
#divide the dataset into independent and dependent set
X=file.drop('Exited',axis=1)
y=file['Exited']

In [22]:
# split in train and test set 
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [23]:
##scaele the features
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [24]:
X_train

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,geo_france,geo_germany,geo_spain
9254,686,1,32,6,0.00,2,1,1,179093.26,1.0,0.0,0.0
1561,632,1,42,4,119624.60,2,1,1,195978.86,0.0,1.0,0.0
1670,559,1,24,3,114739.92,1,1,0,85891.02,0.0,0.0,1.0
6087,561,0,27,9,135637.00,1,1,0,153080.40,1.0,0.0,0.0
6669,517,1,56,9,142147.32,1,0,0,39488.04,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,768,1,54,8,69712.74,1,1,1,69381.05,1.0,0.0,0.0
5191,682,0,58,1,0.00,1,1,1,706.50,1.0,0.0,0.0
5390,735,0,38,1,0.00,3,0,0,92220.12,1.0,0.0,0.0
860,667,1,43,8,190227.46,1,1,0,97508.04,1.0,0.0,0.0


In [25]:
pd.DataFrame(X_train_scaled)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.356500,0.913248,-0.655786,0.345680,-1.218471,0.808436,0.649203,0.974817,1.367670,1.001501,-0.579467,-0.576388
1,-0.203898,0.913248,0.294938,-0.348369,0.696838,0.808436,0.649203,0.974817,1.661254,-0.998501,1.725723,-0.576388
2,-0.961472,0.913248,-1.416365,-0.695393,0.618629,-0.916688,0.649203,-1.025834,-0.252807,-0.998501,-0.579467,1.734942
3,-0.940717,-1.094993,-1.131148,1.386753,0.953212,-0.916688,0.649203,-1.025834,0.915393,1.001501,-0.579467,-0.576388
4,-1.397337,0.913248,1.625953,1.386753,1.057449,-0.916688,-1.540351,-1.025834,-1.059600,1.001501,-0.579467,-0.576388
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,1.207474,0.913248,1.435808,1.039728,-0.102301,-0.916688,0.649203,0.974817,-0.539860,1.001501,-0.579467,-0.576388
7996,0.314989,-1.094993,1.816097,-1.389442,-1.218471,-0.916688,0.649203,0.974817,-1.733882,1.001501,-0.579467,-0.576388
7997,0.865009,-1.094993,-0.085351,-1.389442,-1.218471,2.533560,-1.540351,-1.025834,-0.142765,1.001501,-0.579467,-0.576388
7998,0.159323,0.913248,0.390011,1.039728,1.827259,-0.916688,0.649203,-1.025834,-0.050826,1.001501,-0.579467,-0.576388


In [26]:
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler,f)

In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [28]:
X_train_scaled.shape[1]

12

In [29]:
## build the model 
model = Sequential([
  Dense(64, activation='relu',input_shape=(X_train_scaled.shape[1],)), # input layer 
  Dense(32, activation='relu'), #hl2
  Dense(1,activation='sigmoid') # output layer
])


In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [32]:
##complile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'],run_eagerly=True)

In [33]:
model

In [34]:
## set the Tensorboard 
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir="./logs", write_graph=False)

In [35]:
## set up early stopping
early_stopping_callback= EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True)

In [36]:
print(X_train_scaled.shape, y_train.shape)
print(X_test_scaled.shape, y_test.shape)


(8000, 12) (8000,)
(2000, 12) (2000,)


In [37]:
## training the model 
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=25,
    batch_size=32,
    callbacks=[tensorflow_callback, early_stopping_callback]
)


Epoch 1/25

250/250 [==============================] - 8s 29ms/step - loss: 0.4490 - accuracy: 0.8031 - val_loss: 0.3953 - val_accuracy: 0.8270
Epoch 2/25
250/250 [==============================] - 7s 27ms/step - loss: 0.3869 - accuracy: 0.8378 - val_loss: 0.3586 - val_accuracy: 0.8520
Epoch 3/25
250/250 [==============================] - 7s 28ms/step - loss: 0.3571 - accuracy: 0.8528 - val_loss: 0.3464 - val_accuracy: 0.8605
Epoch 4/25
250/250 [==============================] - 7s 29ms/step - loss: 0.3448 - accuracy: 0.8587 - val_loss: 0.3465 - val_accuracy: 0.8590
Epoch 5/25
250/250 [==============================] - 7s 28ms/step - loss: 0.3392 - accuracy: 0.8587 - val_loss: 0.3403 - val_accuracy: 0.8590
Epoch 6/25
250/250 [==============================] - 7s 27ms/step - loss: 0.3364 - accuracy: 0.8606 - val_loss: 0.3415 - val_accuracy: 0.8605
Epoch 7/25
250/250 [==============================] - 7s 27ms/step - loss: 0.3320 - accuracy: 0.8631 - val_loss: 0.3375 - val_accuracy: 0.862

In [38]:
model.save('model.h5')

e:\machine_learing_course\dl_project_2\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [39]:
##load tensorboard extension 
%load_ext tensorboard

In [43]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 4476), started 0:07:49 ago. (Use '!kill 4476' to kill it.)